In [74]:
from enum import Enum
from functools import total_ordering
from http import HTTPStatus

### customizing

In [2]:
class Color(Enum):
    red = 1
    green = 2
    blue = 3
    
    
    # what is going to be instance of Color? Color.red or Color.blue and so on
    
    def purecolor(self, value):
        return {self:value} # all members of enumerations are hashable

In [3]:
Color.red # instance of Color - default representation

<Color.red: 1>

In [5]:
Color.red.purecolor(100), Color.blue.purecolor(255)

({<Color.red: 1>: 100}, {<Color.blue: 3>: 255})

let's override default representation

In [7]:
class Color(Enum):
    red = 1
    green = 2
    blue = 3
    
    def __repr__(self):
        return f"{self.name} ({self.value})"
    
    def purecolor(self, value):
        return {self:value} 

In [8]:
Color.red

red (1)

#### ordering

In [11]:
class Number(Enum):
    one = 1
    two = 2
    three = 3
Number.one > Number.two

TypeError: '>' not supported between instances of 'Number' and 'Number'

In [12]:
class Number(Enum):
    one = 1
    two = 2
    three = 3
    
    def __lt__(self, other):
        return isinstance(other, Number) and self.value < other.value
Number.one > Number.two

False

In [13]:
Number.one == 1 #we can make it work

False

In [15]:
class Number(Enum):
    one = 1
    two = 2
    three = 3
    
    def __lt__(self, other):
        return isinstance(other, Number) and self.value < other.value
    
    def __eq__(self, other):
        if isinstance(other, Number):
            return self is other
        elif isinstance(other, int):
            return self.value == other
        return False


In [16]:
Number.one == 1, Number.one == Number.one, Number.one is Number.one

(True, True, True)

In [17]:
Number.one == 1.0 

False

but now Number is not hashable - we may need to implement hash

back to ordering

In [18]:
Number.one < Number.two, Number.one > Number.two

(True, False)

In [19]:
Number.one <= Number.two

TypeError: '<=' not supported between instances of 'Number' and 'Number'

In [21]:
@total_ordering
class Number(Enum):
    one = 1
    two = 2
    three = 3
    
    def __lt__(self, other):
        return isinstance(other, Number) and self.value < other.value
    
    def __eq__(self, other):
        if isinstance(other, Number):
            return self is other
        elif isinstance(other, int):
            return self.value == other
        return False

In [22]:
Number.one <= Number.two

True

In [23]:
class Phase(Enum):
    ready = "ready"
    running = "running"
    finished = "finished"
    
    def __str__(self):
        return self.value
    def __eq__(self, other):
        if isinstance(other, Phase):
            return self is other
        elif isinstance(other, str):
            return self.value == other
        return False
    
    def __lt__(self, other):
        ordered_items = list(Phase)
        self_order_index = ordered_items.index(self)
        
        if isinstance(other, Phase):
            other_order_index = ordered_items.index(other)
            return self_order_index < other_order_index
        if isinstance(other, str): 
            try:
                other_member = Phase(other)
                other_order_index = ordered_items.index(other_member)
                return self_order_index < other_order_index
            except ValueError:
                return False
                

In [24]:
Phase.ready == "ready"

True

In [25]:
Phase.ready < Phase.running

True

In [26]:
Phase.ready < "running"

True

#### bool

In [49]:
class State(Enum):
    ready = 1
    busy = 0

In [50]:
bool(State.ready), bool(State.busy)

(True, True)

In [51]:
state = State.busy
if state is State.ready:
    print("ready!")
else:
    print("busy")

busy


In [52]:
state = State.busy
if state:
    print("ready!")
else:
    print("busy")

ready!


In [53]:
class State(Enum):
    ready = 1
    busy = 0
    
    def __bool__(self):
        return bool(self.value)

In [54]:
state = State.busy
if state:
    print("ready!")
else:
    print("busy")

busy


In [56]:
class Dummy(Enum):
    A = 0
    B = 1
    C = ""
    D = "python"
    
    def __bool__(self):
        return bool(self.value)
    

In [57]:
bool(Dummy.A), bool(Dummy.B), bool(Dummy.C), bool(Dummy.D)

(False, True, False, True)

### Extending

In [58]:
class Color(Enum):
    red = 1
    green = 2
    blue = 3
class ColorAlpha(Color):
    pass

TypeError: Cannot extend enumerations

In [59]:
class ColorBase(Enum):
    def hello(self):
        return f"{str(self)} says hello!"

In [60]:
class Color(ColorBase):
    red = "red"
    blue = "blue"
    green = "green"

In [61]:
Color.red.hello()

'Color.red says hello!'

In [69]:
@total_ordering
class OrderedEnum(Enum):
    
    def __lt__(self, other):
         if isinstance(other, OrderedEnum):
            return self.value < other.value
         return NonImplemented
        
class Number(OrderedEnum):
    one = 1
    two = 2
    three = 3
    
class Dimentions(OrderedEnum):
    D1 = 1,
    D2 = 1, 1, 
    D3 = 1, 1, 1

In [72]:
Number.one <= Number.two

True

In [73]:
Number.one < Dimentions.D2

TypeError: '<' not supported between instances of 'int' and 'tuple'

#### HTTPStatus

In [76]:
type(HTTPStatus)

enum.EnumMeta

In [77]:
list(HTTPStatus)

[<HTTPStatus.CONTINUE: 100>,
 <HTTPStatus.SWITCHING_PROTOCOLS: 101>,
 <HTTPStatus.PROCESSING: 102>,
 <HTTPStatus.OK: 200>,
 <HTTPStatus.CREATED: 201>,
 <HTTPStatus.ACCEPTED: 202>,
 <HTTPStatus.NON_AUTHORITATIVE_INFORMATION: 203>,
 <HTTPStatus.NO_CONTENT: 204>,
 <HTTPStatus.RESET_CONTENT: 205>,
 <HTTPStatus.PARTIAL_CONTENT: 206>,
 <HTTPStatus.MULTI_STATUS: 207>,
 <HTTPStatus.ALREADY_REPORTED: 208>,
 <HTTPStatus.IM_USED: 226>,
 <HTTPStatus.MULTIPLE_CHOICES: 300>,
 <HTTPStatus.MOVED_PERMANENTLY: 301>,
 <HTTPStatus.FOUND: 302>,
 <HTTPStatus.SEE_OTHER: 303>,
 <HTTPStatus.NOT_MODIFIED: 304>,
 <HTTPStatus.USE_PROXY: 305>,
 <HTTPStatus.TEMPORARY_REDIRECT: 307>,
 <HTTPStatus.PERMANENT_REDIRECT: 308>,
 <HTTPStatus.BAD_REQUEST: 400>,
 <HTTPStatus.UNAUTHORIZED: 401>,
 <HTTPStatus.PAYMENT_REQUIRED: 402>,
 <HTTPStatus.FORBIDDEN: 403>,
 <HTTPStatus.NOT_FOUND: 404>,
 <HTTPStatus.METHOD_NOT_ALLOWED: 405>,
 <HTTPStatus.NOT_ACCEPTABLE: 406>,
 <HTTPStatus.PROXY_AUTHENTICATION_REQUIRED: 407>,
 <HTTPStatus.

In [78]:
HTTPStatus(400)

<HTTPStatus.BAD_REQUEST: 400>

In [79]:
HTTPStatus.OK, HTTPStatus.OK.name, HTTPStatus.OK.value, 

(<HTTPStatus.OK: 200>, 'OK', 200)

In [80]:
HTTPStatus.NOT_FOUND.value, HTTPStatus.NOT_FOUND.name, HTTPStatus.NOT_FOUND.phrase,

(404, 'NOT_FOUND', 'Not Found')

let's implement phrase

In [92]:
class AppStatus(Enum):
    OK = (0, "No problem!")
    FAILED = (1, 'Crap!')
    

In [93]:
AppStatus.OK

<AppStatus.OK: (0, 'No problem!')>

In [94]:
class AppStatus(Enum):
    OK = (0, "No problem!")
    FAILED = (1, 'Crap!')
    
    @property
    def code(self):
        return self.value[0]
    
    @property
    def phrase(self):
        return self.value[1]

In [95]:
AppStatus.OK.code, AppStatus.OK.phrase, AppStatus.OK.name, 

(0, 'No problem!', 'OK')

In [96]:
HTTPStatus(400)

<HTTPStatus.BAD_REQUEST: 400>

In [97]:
AppStatus(0)

ValueError: 0 is not a valid AppStatus

In [106]:
class AppStatus(Enum):
    OK = (0, "No problem!")
    FAILED = (1, 'Crap!')
    
    def __new__(cls, member_value, member_phrase):
        print(cls, member_value, member_phrase)
        member = object.__new__(cls)
        
        member._value_ = member_value
        member.phrase = member_phrase
        return member
    
    

<enum 'AppStatus'> 0 No problem!
<enum 'AppStatus'> 1 Crap!


In [107]:
AppStatus.OK.value, AppStatus.OK.phrase, AppStatus.OK.name, 

(0, 'No problem!', 'OK')

In [108]:
AppStatus(0), AppStatus(1)

(<AppStatus.OK: 0>, <AppStatus.FAILED: 1>)

In [109]:
AppStatus("No problem!")

ValueError: 'No problem!' is not a valid AppStatus

In [103]:
class TwoValueEnum(Enum):
    def __new__(cls, member_value, member_phrase):
        member = object.__new__(cls)
        
        member._value_ = member_value
        member.phrase = member_phrase
        return member

In [104]:
class AppStatus(TwoValueEnum):
    OK = (0, "No problem!")
    FAILED = (1, 'Crap!')

In [105]:
AppStatus.FAILED.value, AppStatus.FAILED.phrase, AppStatus.FAILED.name, 

(1, 'Crap!', 'FAILED')